## Model trained to detect Open/Closed Eye and Open/Closed Mouth

In [ ]:
import cv2
import torch
from ultralytics import YOLO

model = YOLO("/models/EYE_MOUTH.pt")  # Change to your model path if using YOLOv8 (e.g., "yolov8n.pt")
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)

    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # Bounding box coordinates
            conf = float(box.conf[0])  # Confidence score
            cls = int(box.cls[0])  # Class ID
            label = f"{model.names[cls]}: {conf:.2f}"
            
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 5) 
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 3)  

    cv2.imshow("YOLO Live", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


## Model trained to detect Awake,Drowsy,Yawning

In [ ]:
import cv2
import torch
from ultralytics import YOLO

model = YOLO("/models/drowsiness.pt")  

cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)
    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # Bounding box coordinates
            conf = float(box.conf[0])  # Confidence score
            cls = int(box.cls[0])  # Class ID
            label = f"{model.names[cls]}: {conf:.2f}"

            # Draw thicker rectangle
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 5) 
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 3) 
    cv2.imshow("YOLO Live", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


## Main Model(combined) along with beeps 

In [17]:
import cv2
import torch
import time
import winsound
from ultralytics import YOLO

model1 = YOLO("/models/EYE_MOUTH.pt")  # Model 1 (e.g., Eye detection)
model2 = YOLO("/models/drowsiness.pt")  # Model 2 (e.g., Mouth detection)
cap = cv2.VideoCapture(0)

closed_eye_start = None
fatigue_threshold = 3  

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run both models
    results1 = model1(frame)
    results2 = model2(frame)

    detected_labels_1 = set()
    detected_labels_2 = set()

    # Function to draw detections
    def draw_detections(results, frame, color, model, detected_labels):
        for result in results:
            for box in result.boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                conf = float(box.conf[0])
                cls = int(box.cls[0])
                
                if cls in model.names:
                    label = model.names[cls]
                    detected_labels.add(label)

                    cv2.rectangle(frame, (x1, y1), (x2, y2), color, 3)
                    cv2.putText(frame, f"{label}: {conf:.2f}", (x1, y1 - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

    # Draw detections from both models
    draw_detections(results1, frame, (0, 255, 0), model1, detected_labels_1)  # Green for Model 1
    draw_detections(results2, frame, (255, 0, 0), model2, detected_labels_2)  # Blue for Model 2

    # Fatigue Detection Logic
    current_time = time.time()

    # If eyes are closed
    if "closed_eye" in detected_labels_1:
        if closed_eye_start is None:
            closed_eye_start = current_time
        elif current_time - closed_eye_start > fatigue_threshold:
            cv2.putText(frame, "WARNING: DROWSINESS DETECTED!", (50, 50), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
            winsound.Beep(1000, 500)  # Beep sound for alert
    else:
        closed_eye_start = None  # Reset if eyes are open

    # If yawning is detected
    if "Yawning" in detected_labels_2:
        cv2.putText(frame, "WARNING: YAWNING DETECTED!", (50, 100), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
        winsound.Beep(1500, 700)  # Higher frequency beep

    # If drowsy is detected from Model 2
    if "Drowsy" in detected_labels_2:
        cv2.putText(frame, "ALERT: DRIVER DROWSY!", (50, 150), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 3)
        winsound.Beep(2000, 1000)  # Long alert beep

    # If both closed eyes and closed mouth for long → High fatigue
    if "closed_eye" in detected_labels_1 and "open_mouth" in detected_labels_1:
        if closed_eye_start is not None and current_time - closed_eye_start > fatigue_threshold:
            cv2.putText(frame, "ALERT: EXTREME FATIGUE DETECTED!", (50, 200), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
            winsound.Beep(2500, 1200)  # Long fatigue alert

    # Show the frame
    cv2.imshow("Fatigue Detection", frame)

    # Exit on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()



0: 480x640 (no detections), 74.4ms
Speed: 2.0ms preprocess, 74.4ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Awake, 79.1ms
Speed: 2.2ms preprocess, 79.1ms inference, 5.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 open_eye, 1 open_mouth, 59.5ms
Speed: 2.0ms preprocess, 59.5ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Awake, 56.7ms
Speed: 0.9ms preprocess, 56.7ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 closed_eye, 1 open_eye, 1 open_mouth, 56.2ms
Speed: 1.0ms preprocess, 56.2ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Awake, 47.7ms
Speed: 0.0ms preprocess, 47.7ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 closed_eye, 1 open_mouth, 46.0ms
Speed: 1.0ms preprocess, 46.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Awake, 45.8ms
Speed: 1.4ms preproces

Speed: 1.0ms preprocess, 48.6ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Awake, 46.3ms
Speed: 1.0ms preprocess, 46.3ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 closed_eye, 1 open_mouth, 45.9ms
Speed: 1.0ms preprocess, 45.9ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Awake, 45.2ms
Speed: 1.0ms preprocess, 45.2ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 closed_mouth, 1 open_eye, 1 open_mouth, 45.3ms
Speed: 1.0ms preprocess, 45.3ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Awake, 46.0ms
Speed: 1.5ms preprocess, 46.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 closed_mouth, 1 open_eye, 1 open_mouth, 45.7ms
Speed: 2.0ms preprocess, 45.7ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Awake, 46.4ms
Speed: 1.0ms preprocess, 46.4ms inferenc


0: 480x640 1 Awake, 45.5ms
Speed: 1.0ms preprocess, 45.5ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 closed_eye, 1 closed_mouth, 1 open_eye, 1 open_mouth, 45.0ms
Speed: 1.0ms preprocess, 45.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Awake, 45.6ms
Speed: 1.0ms preprocess, 45.6ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 closed_eye, 1 closed_mouth, 1 open_mouth, 46.5ms
Speed: 0.0ms preprocess, 46.5ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Awake, 45.2ms
Speed: 1.0ms preprocess, 45.2ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 closed_eye, 1 closed_mouth, 1 open_mouth, 47.5ms
Speed: 1.0ms preprocess, 47.5ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Awake, 46.0ms
Speed: 1.0ms preprocess, 46.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x


0: 480x640 1 closed_eye, 1 closed_mouth, 1 open_mouth, 45.0ms
Speed: 1.0ms preprocess, 45.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Awake, 46.4ms
Speed: 1.2ms preprocess, 46.4ms inference, 2.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 closed_eye, 1 closed_mouth, 1 open_mouth, 45.0ms
Speed: 1.0ms preprocess, 45.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Awake, 45.4ms
Speed: 1.1ms preprocess, 45.4ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 closed_eye, 1 closed_mouth, 45.0ms
Speed: 1.0ms preprocess, 45.0ms inference, 2.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Awake, 46.7ms
Speed: 0.0ms preprocess, 46.7ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 closed_eye, 1 closed_mouth, 45.6ms
Speed: 1.0ms preprocess, 45.6ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1


0: 480x640 1 Awake, 45.6ms
Speed: 1.0ms preprocess, 45.6ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 open_mouth, 45.6ms
Speed: 0.9ms preprocess, 45.6ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Awake, 45.3ms
Speed: 2.2ms preprocess, 45.3ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 open_eye, 1 open_mouth, 45.5ms
Speed: 1.0ms preprocess, 45.5ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Awake, 48.3ms
Speed: 1.0ms preprocess, 48.3ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 open_mouth, 45.4ms
Speed: 1.0ms preprocess, 45.4ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Awake, 45.6ms
Speed: 2.0ms preprocess, 45.6ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 closed_mouth, 1 open_mouth, 45.9ms
Speed: 0.9ms preprocess, 45.9ms inference, 1.0ms 


0: 480x640 1 open_mouth, 45.9ms
Speed: 1.0ms preprocess, 45.9ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Awake, 46.6ms
Speed: 1.0ms preprocess, 46.6ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 open_mouth, 46.5ms
Speed: 1.0ms preprocess, 46.5ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Awake, 45.4ms
Speed: 2.3ms preprocess, 45.4ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 open_mouth, 46.1ms
Speed: 1.0ms preprocess, 46.1ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Awake, 45.5ms
Speed: 1.0ms preprocess, 45.5ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 open_eye, 1 open_mouth, 44.8ms
Speed: 1.0ms preprocess, 44.8ms inference, 1.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Awake, 45.1ms
Speed: 2.0ms preprocess, 45.1ms inference, 1.0ms postprocess per 

In [19]:
import cv2
import torch
import ultralytics
import numpy as np

print("OpenCV Version:", cv2.__version__)
print("Torch Version:", torch.__version__)
print("Ultralytics Version:", ultralytics.__version__)
print("NumPy Version:", np.__version__)


OpenCV Version: 4.11.0
Torch Version: 2.5.1+cu124
Ultralytics Version: 8.3.63
NumPy Version: 1.24.3
